In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score,roc_curve,auc

import toad
from sklearn.linear_model import LogisticRegression

In [2]:
file1 = pd.read_excel(r'4.2转化后数据.xlsx')
file1.head()

,文件,罪名,主刑,时长,性别,学历,前科次数,自首,坦白,立功,...,偶犯,初犯,中止,未遂,"情节, 严重","情节, 轻","社会, 危害性, 严重",恶劣,"被害人, 过错",判决时被告人年龄
0,GYSR_0001,故意杀人,无期徒刑,NaN,1,3.0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,30.0
1,GYSR_0002,故意杀人,无期徒刑,NaN,1,2.0,2,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,37.0
2,GYSR_0003,故意杀人,有期徒刑,十五年,1,1.0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0
3,GYSR_0004,故意杀人,无期徒刑,NaN,0,1.0,0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0
4,GYSR_0005,故意杀人,有期徒刑,五年六个月,1,2.0,0,1.0,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,27.0


In [3]:
file1.shape

(890, 24)

In [4]:
tmplist = ['故意杀人', '故意杀人（未遂）', '故意杀人（中止）']
file1 = file1[file1['罪名'].isin(tmplist) == True]
file1.shape

(861, 24)

In [5]:
tmplist = ['死刑', '无期徒刑', '有期徒刑']
file1 = file1[file1['主刑'].isin(tmplist) == True]
file1.shape

(855, 24)

In [6]:
mapping = {'死刑': 1, '无期徒刑': 1, '有期徒刑': 0}
file1['Y'] = file1['主刑'].map(mapping)
file1.head()

,文件,罪名,主刑,时长,性别,学历,前科次数,自首,坦白,立功,...,初犯,中止,未遂,"情节, 严重","情节, 轻","社会, 危害性, 严重",恶劣,"被害人, 过错",判决时被告人年龄,Y
0,GYSR_0001,故意杀人,无期徒刑,NaN,1,3.0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,30.0,1
1,GYSR_0002,故意杀人,无期徒刑,NaN,1,2.0,2,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,37.0,1
2,GYSR_0003,故意杀人,有期徒刑,十五年,1,1.0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,0
3,GYSR_0004,故意杀人,无期徒刑,NaN,0,1.0,0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0,1
4,GYSR_0005,故意杀人,有期徒刑,五年六个月,1,2.0,0,1.0,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,27.0,0


In [7]:
del file1['罪名']
del file1['主刑']
del file1['时长']
file1.head()

,文件,性别,学历,前科次数,自首,坦白,立功,认罪,认罚,悔罪,...,初犯,中止,未遂,"情节, 严重","情节, 轻","社会, 危害性, 严重",恶劣,"被害人, 过错",判决时被告人年龄,Y
0,GYSR_0001,1,3.0,0,NaN,NaN,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,30.0,1
1,GYSR_0002,1,2.0,2,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,37.0,1
2,GYSR_0003,1,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,0
3,GYSR_0004,0,1.0,0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0,1
4,GYSR_0005,1,2.0,0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,27.0,0


In [8]:
y = 'Y'

In [9]:
# 随机划分训练集和测试集
import random
def random_split(df, train_rate):
    train_list = random.sample(range(0, len(df)), int(len(df) * train_rate))
    test_list = list(set(range(0, len(df))) - set(train_list))
    train_df = df[df.index.isin(train_list) == True]
    test_df = df[df.index.isin(test_list) == True]
    return train_df, test_df
train_df, test_df = random_split(file1, 0.7)

In [10]:
train_df.head()

,文件,性别,学历,前科次数,自首,坦白,立功,认罪,认罚,悔罪,...,初犯,中止,未遂,"情节, 严重","情节, 轻","社会, 危害性, 严重",恶劣,"被害人, 过错",判决时被告人年龄,Y
0,GYSR_0001,1,3.0,0,NaN,NaN,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,30.0,1
2,GYSR_0003,1,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,0
3,GYSR_0004,0,1.0,0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0,1
4,GYSR_0005,1,2.0,0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,27.0,0
5,GYSR_0006,1,2.0,0,1.0,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,39.0,0


In [11]:
train_df.shape

(571, 22)

In [12]:
test_df.shape

(249, 22)

In [13]:
feature_list = train_df.drop(['文件', y], axis=1).columns.to_list()
len(feature_list)

20

In [14]:
# toad分箱
c=toad.transform.Combiner()
c.fit(file1[feature_list+[y]],y=y,method='chi',min_samples=0.1,empty_separate=True,exclude=[])

In [15]:
dict_1 = c.export()
dict_1

{'性别': [1],
 '学历': [2.0, nan],
 '前科次数': [],
 '自首': [1.0, nan],
 '坦白': [1.0, nan],
 '立功': [nan],
 '认罪': [nan],
 '认罚': [nan],
 '悔罪': [nan],
 '谅解': [nan],
 '偶犯': [nan],
 '初犯': [1.0, nan],
 '中止': [1.0, nan],
 '未遂': [nan],
 '情节, 严重': [1.0, nan],
 '情节, 轻': [1.0, nan],
 '社会, 危害性, 严重': [1.0, nan],
 '恶劣': [1.0, nan],
 '被害人, 过错': [1.0, nan],
 '判决时被告人年龄': [30.0, 48.0, nan]}

In [16]:
#对变量进行woe转换
t = toad.transform.WOETransformer() 
train_woe = t.fit_transform(c.transform(train_df[feature_list+[y]]), train_df[y], exclude=[y])
test_woe = t.transform(c.transform(test_df[feature_list+[y]]))

In [17]:
# 逻辑回归算法
lr = LogisticRegression(penalty='l2', solver='liblinear', C=0.01, random_state=0, verbose=2)
lr.fit(train_woe[feature_list], train_woe[y])

[LibLinear]

LogisticRegression(C=0.01, random_state=0, solver='liblinear', verbose=2)

In [18]:
#计算train样本的auc,ks
y_train_pred = lr.predict_proba(train_woe[feature_list])[:,1]
fpr_train, tpr_train, threshold_train = roc_curve(train_woe[y], y_train_pred, drop_intermediate=False)
auc_train = roc_auc_score(train_woe[y], y_train_pred)
ks_train = max(abs(tpr_train - fpr_train) * 100)

In [19]:
y_test_pred = lr.predict_proba(test_woe[feature_list])[:,1]
fpr_test, tpr_test, threshold_test = roc_curve(test_woe[y], y_test_pred, drop_intermediate=False)
auc_test = roc_auc_score(test_woe[y], y_test_pred)
ks_test = max(abs(tpr_test - fpr_test) * 100)

In [20]:
print('train ks:{}, auc:{}: '.format(ks_train, auc_train))
print('test ks:{}, auc:{}: '.format(ks_test, auc_test))

train ks:53.97717511351085, auc:0.8442017425450975: 
test ks:50.974193548387106, auc:0.8311612903225807: 


In [21]:
#将变量系数与常数保存下来
col_coef=[]
for i in lr.coef_:
    for j in i:
        col_coef.append(j)
col_coef=pd.DataFrame({'feature':feature_list,'coef':col_coef})
col_coef

,feature,coef
0,性别,0.170987
1,学历,0.032411
2,前科次数,0.000000
3,自首,0.001979
4,坦白,0.020031
5,立功,-0.001648
6,认罪,0.000056
7,认罚,0.010735
8,悔罪,0.015813
9,谅解,0.255474
